In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji

     |████████████████████████████████| 170 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=1b8728e03b7dcba6dc9b57b8b07ed31a4c6356822ecef799fc04fae82b2a2670
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 94700, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 94700 (delta 0), reused 1 (delta 0), pack-reused 94697
Receiving objects: 100% (94700/94700), 79.91 MiB | 20.57 MiB/s, done.
Resolving deltas: 100% (68764/68764), done.


In [ ]:
%cd transformers

/content/transformers


In [ ]:
!pip3 install --upgrade .

Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 5.4 MB/s 
     |████████████████████████████████| 6.8 MB 41.4 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
     |████████████████████████████████| 61 kB 481 kB/s 
  Created wheel for transformers: filename=transformers-4.16.0.dev0-py3-none-any.whl size=3377760 sha256=1c6c0a916311c03f9e74b0454237f149b7b06ec9b8fb34b96475df86f565ec81
  Stored in directory: /tmp/pip-ephem-wheel-ca

In [ ]:
import os
import json
import random
import time
import datetime
import torch
import argparse
import numpy as np
import pandas as pd
from torch.nn import functional as F
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix
from transformers import get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler, Dataset

In [ ]:
batch_size = 4

In [ ]:
def convert_label(label):
    if label == "desirability":
        return 0
    elif label == "enabling context":
        return 1
    elif label == "can do":
        return 2
    elif label == "buzz":
        return 3
    elif label == "invitation":
        return 4
    else:
        raise Exception("label classes must be 'desirability' or 'enabling context' or 'buzz' or 'can do' or 'invitation'")

In [ ]:
%cd ..

/content


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Research Track/Dataset/Experiment Dataset/Dataset_V4.csv')
df

,Unnamed: 0,decision,created_at,id,id_str,full_text,source,retweet_count,favorite_count,reply,username_length,user_location
0,0,enabling context,2021-01-30 02:40:25+00:00,1355345090022154241,1355345090022154241,@science_barbie @priyology Cyanobacteria (also...,Twitter Web App,0,0,True,8,Unknown
1,1,enabling context,2021-01-30 01:05:16+00:00,1355321147701686273,1355321147701686273,New insights into coral bleaching by internati...,Twitter Web App,1,1,False,10,Unknown
2,2,buzz,2021-01-29 22:49:17+00:00,1355286925809094659,1355286925809094659,"Friday Tally:\n\n2 dives, continued coral blea...",Twitter for iPhone,0,1,False,11,Unknown
3,3,desirability,2021-01-29 22:09:20+00:00,1355276872410681348,1355276872410681348,Coral bleaching is so sad because they just be...,Twitter for iPhone,0,2,False,8,Unknown
4,4,desirability,2021-01-29 21:34:44+00:00,1355268165148815360,1355268165148815360,predicts that at the current rate of emissions...,Twitter Web App,0,0,True,13,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
1217,1217,buzz,2021-05-17 19:30:12+00:00,1394374712835264513,1394374712835264513,#ClimateChampions developed learning resources...,Twitter Web App,4,14,False,10,United States
1218,1218,desirability,2021-05-17 17:37:33+00:00,1394346363736018949,1394346363736018949,"""For all we hear about #coral bleaching &amp; ...",Twitter Web App,11,24,False,14,Unknown
1219,1219,enabling context,2021-05-17 17:01:39+00:00,1394337327930122250,1394337327930122250,Corals are critical. Track coral bleaching in ...,Hootsuite Inc.,10,29,False,13,United States
1220,1220,enabling context,2021-05-17 16:09:02+00:00,1394324089200779268,1394324089200779268,The number one root cause of coral reefs facin...,Twitter for iPhone,1,0,True,12,South Korea


In [ ]:
df = df[['decision','full_text']]
df = df.rename(columns={'full_text':'text','decision':'label'})

In [ ]:
df['label'] = df['label'].map({
    'desirability': 0, 
    'enabling context': 1,
    'can do': 2,
    'buzz':3,
    'invitation': 4
    })

In [ ]:
df['label'] = df['label'].astype(int)

In [ ]:
df

,label,text
0,1,@science_barbie @priyology Cyanobacteria (also...
1,1,New insights into coral bleaching by internati...
2,3,"Friday Tally:\n\n2 dives, continued coral blea..."
3,0,Coral bleaching is so sad because they just be...
4,0,predicts that at the current rate of emissions...
...,...,...
1217,3,#ClimateChampions developed learning resources...
1218,0,"""For all we hear about #coral bleaching &amp; ..."
1219,1,Corals are critical. Track coral bleaching in ...
1220,1,The number one root cause of coral reefs facin...


In [ ]:
def cleansing(text, stopword = None):
    # remove hastag
    text = ' '.join(word for word in text.split(' ') if not word.startswith('#'))
    # remove mention
    text = ' '.join(word for word in text.split(' ') if not word.startswith('@'))
    # remove URL
    text = ' '.join(word for word in text.split(' ') if not word.startswith('https:'))

    # case folding to lowercase
    text = text.lower()
    # split by space
    word_list = text.split()
    # pick only word with length greater than 2
    word_list = [word for word in word_list if len(word) > 2]
    # pick only alpha-numeric word 
    word_list = [word for word in word_list if word.isalnum()]

    if stopword == None:
        text = ' '.join(word_list)
    else:
        word_list = [word for word in word_list if word not in stopword]
        text = ' '.join(word_list)
                
    return text

In [ ]:
for i in range(len(df)):
    df.loc[i, 'text'] = cleansing(df.loc[i, 'text'])

In [ ]:
df

,label,text
0,1,cyanobacteria called are microorganisms that c...
1,1,new insights into coral bleaching internationa...
2,3,friday continued coral bleaching documentation...
3,0,coral bleaching sad because they just there vi...
4,0,predicts that the current rate there likely in...
...,...,...
1217,3,developed learning resources and curriculum fo...
1218,0,all hear about bleaching other early these imp...
1219,1,corals are track coral bleaching the pacific w...
1220,1,the number one root cause coral reefs facing d...


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer 

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
features_list = []
for i in range(0,len(df)):
  line = df['text'][i]
  input_ids = torch.tensor([tokenizer.encode(line)])
  with torch.no_grad():
    features = bertweet(input_ids)  # Models outputs are now tuples
    features_list.append(features)

In [ ]:
f_l = []
for i in range(0,len(features_list)):
  fl = features_list[i].pooler_output.numpy()
  f_l.append(fl)

In [ ]:
len(f_l)

1222

In [ ]:
from pandas import DataFrame
feature_list_new = []
for i in range(0,len(f_l)):
  feature_list_new.append(f_l[i][0])

In [ ]:
from pandas import DataFrame

new_df = DataFrame(feature_list_new)
new_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.269467,-0.237420,0.135789,-0.192115,0.092847,-0.209697,0.151208,0.013293,0.062865,-0.040700,-0.029866,-0.065176,-0.253705,0.151779,0.113438,-0.026016,-0.196706,0.011119,0.023070,0.025161,-0.181920,-0.156438,0.293603,0.047054,-0.022497,0.095719,-0.257094,-0.014528,-0.010651,0.068333,-0.049718,-0.232437,0.338180,0.261451,0.188081,-0.174729,0.128956,0.033906,0.066476,0.123503,...,0.177455,-0.184963,-0.146089,0.190156,0.335575,-0.025112,0.119993,0.064195,-0.212036,0.042518,-0.273705,-0.080256,-0.197115,-0.060150,-0.168341,0.013836,-0.037829,0.069539,-0.080841,0.045633,-0.299846,0.345259,-0.188063,-0.256294,0.115115,0.344267,-0.073296,-0.155676,0.168050,0.128368,-0.087861,-0.377813,-0.091366,0.112518,0.142527,0.185022,-0.214439,0.012216,-0.082393,-0.240312
1,0.278989,-0.210553,0.116534,-0.220855,0.026345,-0.185116,0.102590,-0.142175,0.066128,-0.152199,-0.003948,-0.127940,-0.262440,0.150455,0.154574,-0.015205,-0.147123,0.033080,0.117147,0.082415,-0.193111,-0.194614,0.289233,-0.045297,0.034218,0.129791,-0.166426,-0.094949,0.109077,-0.081011,-0.022905,-0.255659,0.114677,0.213910,0.165936,-0.204131,0.134564,0.064170,-0.019278,0.051917,...,-0.035442,-0.165967,-0.130318,0.158661,0.368667,-0.099549,0.056852,0.038233,-0.229869,0.054649,-0.280238,-0.200504,-0.132204,0.040076,-0.213502,-0.052318,-0.019564,0.085647,0.006973,0.123810,-0.326739,0.272688,-0.123412,-0.134248,0.159430,0.330009,0.030360,-0.118556,0.050030,0.047144,-0.147236,-0.331968,-0.149588,0.110895,0.127745,0.284505,-0.328791,0.033071,-0.195695,-0.169238
2,0.292273,-0.185109,0.103384,-0.195429,0.099492,-0.132130,0.051576,-0.111612,0.108165,-0.140106,0.050763,-0.107536,-0.202583,0.039158,0.162155,-0.040363,-0.202544,-0.043050,0.133335,0.085618,-0.123567,-0.227499,0.301947,-0.018139,-0.006174,0.150228,-0.108965,-0.049512,0.111773,-0.091334,0.001697,-0.189163,0.086513,0.200552,0.130424,-0.175857,0.161722,0.030261,-0.055439,-0.013702,...,-0.027425,-0.182329,-0.229967,0.148397,0.412870,-0.160140,0.041154,0.043219,-0.246439,0.076627,-0.274997,-0.134232,-0.117466,0.048612,-0.157647,0.045113,-0.059363,0.082873,-0.038386,0.259776,-0.338416,0.241494,-0.088733,-0.111480,0.178569,0.369975,-0.070284,-0.110966,0.084464,0.102760,-0.082984,-0.293506,-0.167755,0.122283,0.081358,0.308439,-0.346580,-0.050084,-0.225804,-0.227144
3,0.196565,-0.253519,-0.023528,-0.191615,0.086647,-0.220587,-0.055594,-0.048283,0.131703,-0.094454,-0.040577,-0.078372,-0.108059,0.052253,0.228489,-0.088540,-0.319645,-0.121290,0.046943,0.023376,-0.061790,-0.173290,0.351315,-0.008922,0.000887,0.142431,-0.187891,-0.083845,0.031222,-0.001472,-0.017065,-0.215411,0.114584,0.153335,0.115404,-0.104232,0.172777,0.120475,-0.053378,-0.057836,...,0.102314,-0.212893,-0.234050,0.184041,0.429846,-0.131488,0.098180,0.035671,-0.210794,-0.004706,-0.318188,-0.141389,-0.153371,-0.012531,-0.228365,0.001537,-0.116103,0.064897,0.008819,0.117035,-0.350797,0.220972,-0.163182,-0.190531,0.127048,0.421333,-0.146783,-0.109759,0.097357,0.102098,-0.064166,-0.323645,-0.150754,0.236022,0.195547,0.280790,-0.347566,-0.112507,-0.190743,-0.159941
4,0.327024,-0.162196,0.143518,-0.192900,0.083366,-0.133948,0.095954,-0.087004,0.059492,-0.124035,0.145824,-0.162009,-0.277149,0.157513,0.132287,-0.018880,-0.231086,-0.015312,0.149329,0.040641,-0.112284,-0.197858,0.296189,0.010211,0.124315,0.120447,-0.179352,-0.105709,0.115390,-0.032078,-0.075978,-0.162349,0.165580,0.177975,0.163704,-0.180910,0.082442,-0.008368,0.039934,0.052441,...,0.072123,-0.147886,-0.153356,0.123537,0.425206,-0.086512,0.021996,0.053116,-0.244984,0.018976,-0.294460,-0.099158,-0.194668,0.046652,-0.308757,0.006832,-0.028352,-0.083012,-0.078689,0.221611,-0.303508,0.343517,-0.009814,-0.202835,0.162696,0.347850,0.041460,-0.1288

In [ ]:
df_final = pd.concat([df,new_df],axis=1)

In [ ]:
df_final

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,1,cyanobacteria called are microorganisms that c...,0.269467,-0.237420,0.135789,-0.192115,0.092847,-0.209697,0.151208,0.013293,0.062865,-0.040700,-0.029866,-0.065176,-0.253705,0.151779,0.113438,-0.026016,-0.196706,0.011119,0.023070,0.025161,-0.181920,-0.156438,0.293603,0.047054,-0.022497,0.095719,-0.257094,-0.014528,-0.010651,0.068333,-0.049718,-0.232437,0.338180,0.261451,0.188081,-0.174729,0.128956,0.033906,...,0.177455,-0.184963,-0.146089,0.190156,0.335575,-0.025112,0.119993,0.064195,-0.212036,0.042518,-0.273705,-0.080256,-0.197115,-0.060150,-0.168341,0.013836,-0.037829,0.069539,-0.080841,0.045633,-0.299846,0.345259,-0.188063,-0.256294,0.115115,0.344267,-0.073296,-0.155676,0.168050,0.128368,-0.087861,-0.377813,-0.091366,0.112518,0.142527,0.185022,-0.214439,0.012216,-0.082393,-0.240312
1,1,new insights into coral bleaching internationa...,0.278989,-0.210553,0.116534,-0.220855,0.026345,-0.185116,0.102590,-0.142175,0.066128,-0.152199,-0.003948,-0.127940,-0.262440,0.150455,0.154574,-0.015205,-0.147123,0.033080,0.117147,0.082415,-0.193111,-0.194614,0.289233,-0.045297,0.034218,0.129791,-0.166426,-0.094949,0.109077,-0.081011,-0.022905,-0.255659,0.114677,0.213910,0.165936,-0.204131,0.134564,0.064170,...,-0.035442,-0.165967,-0.130318,0.158661,0.368667,-0.099549,0.056852,0.038233,-0.229869,0.054649,-0.280238,-0.200504,-0.132204,0.040076,-0.213502,-0.052318,-0.019564,0.085647,0.006973,0.123810,-0.326739,0.272688,-0.123412,-0.134248,0.159430,0.330009,0.030360,-0.118556,0.050030,0.047144,-0.147236,-0.331968,-0.149588,0.110895,0.127745,0.284505,-0.328791,0.033071,-0.195695,-0.169238
2,3,friday continued coral bleaching documentation...,0.292273,-0.185109,0.103384,-0.195429,0.099492,-0.132130,0.051576,-0.111612,0.108165,-0.140106,0.050763,-0.107536,-0.202583,0.039158,0.162155,-0.040363,-0.202544,-0.043050,0.133335,0.085618,-0.123567,-0.227499,0.301947,-0.018139,-0.006174,0.150228,-0.108965,-0.049512,0.111773,-0.091334,0.001697,-0.189163,0.086513,0.200552,0.130424,-0.175857,0.161722,0.030261,...,-0.027425,-0.182329,-0.229967,0.148397,0.412870,-0.160140,0.041154,0.043219,-0.246439,0.076627,-0.274997,-0.134232,-0.117466,0.048612,-0.157647,0.045113,-0.059363,0.082873,-0.038386,0.259776,-0.338416,0.241494,-0.088733,-0.111480,0.178569,0.369975,-0.070284,-0.110966,0.084464,0.102760,-0.082984,-0.293506,-0.167755,0.122283,0.081358,0.308439,-0.346580,-0.050084,-0.225804,-0.227144
3,0,coral bleaching sad because they just there vi...,0.196565,-0.253519,-0.023528,-0.191615,0.086647,-0.220587,-0.055594,-0.048283,0.131703,-0.094454,-0.040577,-0.078372,-0.108059,0.052253,0.228489,-0.088540,-0.319645,-0.121290,0.046943,0.023376,-0.061790,-0.173290,0.351315,-0.008922,0.000887,0.142431,-0.187891,-0.083845,0.031222,-0.001472,-0.017065,-0.215411,0.114584,0.153335,0.115404,-0.104232,0.172777,0.120475,...,0.102314,-0.212893,-0.234050,0.184041,0.429846,-0.131488,0.098180,0.035671,-0.210794,-0.004706,-0.318188,-0.141389,-0.153371,-0.012531,-0.228365,0.001537,-0.116103,0.064897,0.008819,0.117035,-0.350797,0.220972,-0.163182,-0.190531,0.127048,0.421333,-0.146783,-0.109759,0.097357,0.102098,-0.064166,-0.323645,-0.150754,0.236022,0.195547,0.280790,-0.347566,-0.112507,-0.190743,-0.159941
4,0,predicts that the current rate there likely in...,0.327024,-0.162196,0.143518,-0.192900,0.083366,-0.133948,0.095954,-0.087004,0.059492,-0.124035,0.145824,-0.162009,-0.277149,0.157513,0.132287,-0.018880,-0.231086,-0.015312,0.149329,0.040641,-0.112284,-0.197858,0.296189,0.010211,0.124315,0.120447,-0.179352,-0.105709,0.115390,-0.032078,-0.075978,-0.162349,0.165580,0.177975,0.163704,-0.180910,0.082442,-0.008368,...,0.072123,-0.147886,-0.153356,0.123537,0.425206,-0.086512,0.021996,0.053116,-0.244984,0.018976,

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 5):

    sentences = df_final['text']

    X = feature_list_new

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    df_final['cluster'] = pd.Series(assigned_clusters, index=data.index)
    df_final['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

In [ ]:
clustering_question(df_final)

(      label  ...                                           centroid
 0         1  ...  [0.26434752, -0.2272363, 0.108652465, -0.19666...
 1         1  ...  [0.27022475, -0.20869835, 0.082921416, -0.2030...
 2         3  ...  [0.27022475, -0.20869835, 0.082921416, -0.2030...
 3         0  ...  [0.24856506, -0.19633645, 0.06935101, -0.20951...
 4         0  ...  [0.27742666, -0.17560728, 0.10773915, -0.19093...
 ...     ...  ...                                                ...
 1217      3  ...  [0.23149328, -0.21284643, 0.07621303, -0.18721...
 1218      0  ...  [0.23149328, -0.21284643, 0.07621303, -0.18721...
 1219      1  ...  [0.26434752, -0.2272363, 0.108652465, -0.19666...
 1220      1  ...  [0.27742666, -0.17560728, 0.10773915, -0.19093...
 1221      1  ...  [0.27742666, -0.17560728, 0.10773915, -0.19093...
 
 [1222 rows x 772 columns],
 [4,
  3,
  3,
  2,
  1,
  1,
  2,
  3,
  3,
  0,
  1,
  0,
  3,
  3,
  1,
  0,
  0,
  3,
  1,
  1,
  3,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  

In [ ]:
df_final

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
0,1,cyanobacteria called are microorganisms that c...,0.269467,-0.237420,0.135789,-0.192115,0.092847,-0.209697,0.151208,0.013293,0.062865,-0.040700,-0.029866,-0.065176,-0.253705,0.151779,0.113438,-0.026016,-0.196706,0.011119,0.023070,0.025161,-0.181920,-0.156438,0.293603,0.047054,-0.022497,0.095719,-0.257094,-0.014528,-0.010651,0.068333,-0.049718,-0.232437,0.338180,0.261451,0.188081,-0.174729,0.128956,0.033906,...,-0.146089,0.190156,0.335575,-0.025112,0.119993,0.064195,-0.212036,0.042518,-0.273705,-0.080256,-0.197115,-0.060150,-0.168341,0.013836,-0.037829,0.069539,-0.080841,0.045633,-0.299846,0.345259,-0.188063,-0.256294,0.115115,0.344267,-0.073296,-0.155676,0.168050,0.128368,-0.087861,-0.377813,-0.091366,0.112518,0.142527,0.185022,-0.214439,0.012216,-0.082393,-0.240312,4,"[0.26434752, -0.2272363, 0.108652465, -0.19666..."
1,1,new insights into coral bleaching internationa...,0.278989,-0.210553,0.116534,-0.220855,0.026345,-0.185116,0.102590,-0.142175,0.066128,-0.152199,-0.003948,-0.127940,-0.262440,0.150455,0.154574,-0.015205,-0.147123,0.033080,0.117147,0.082415,-0.193111,-0.194614,0.289233,-0.045297,0.034218,0.129791,-0.166426,-0.094949,0.109077,-0.081011,-0.022905,-0.255659,0.114677,0.213910,0.165936,-0.204131,0.134564,0.064170,...,-0.130318,0.158661,0.368667,-0.099549,0.056852,0.038233,-0.229869,0.054649,-0.280238,-0.200504,-0.132204,0.040076,-0.213502,-0.052318,-0.019564,0.085647,0.006973,0.123810,-0.326739,0.272688,-0.123412,-0.134248,0.159430,0.330009,0.030360,-0.118556,0.050030,0.047144,-0.147236,-0.331968,-0.149588,0.110895,0.127745,0.284505,-0.328791,0.033071,-0.195695,-0.169238,3,"[0.27022475, -0.20869835, 0.082921416, -0.2030..."
2,3,friday continued coral bleaching documentation...,0.292273,-0.185109,0.103384,-0.195429,0.099492,-0.132130,0.051576,-0.111612,0.108165,-0.140106,0.050763,-0.107536,-0.202583,0.039158,0.162155,-0.040363,-0.202544,-0.043050,0.133335,0.085618,-0.123567,-0.227499,0.301947,-0.018139,-0.006174,0.150228,-0.108965,-0.049512,0.111773,-0.091334,0.001697,-0.189163,0.086513,0.200552,0.130424,-0.175857,0.161722,0.030261,...,-0.229967,0.148397,0.412870,-0.160140,0.041154,0.043219,-0.246439,0.076627,-0.274997,-0.134232,-0.117466,0.048612,-0.157647,0.045113,-0.059363,0.082873,-0.038386,0.259776,-0.338416,0.241494,-0.088733,-0.111480,0.178569,0.369975,-0.070284,-0.110966,0.084464,0.102760,-0.082984,-0.293506,-0.167755,0.122283,0.081358,0.308439,-0.346580,-0.050084,-0.225804,-0.227144,3,"[0.27022475, -0.20869835, 0.082921416, -0.2030..."
3,0,coral bleaching sad because they just there vi...,0.196565,-0.253519,-0.023528,-0.191615,0.086647,-0.220587,-0.055594,-0.048283,0.131703,-0.094454,-0.040577,-0.078372,-0.108059,0.052253,0.228489,-0.088540,-0.319645,-0.121290,0.046943,0.023376,-0.061790,-0.173290,0.351315,-0.008922,0.000887,0.142431,-0.187891,-0.083845,0.031222,-0.001472,-0.017065,-0.215411,0.114584,0.153335,0.115404,-0.104232,0.172777,0.120475,...,-0.234050,0.184041,0.429846,-0.131488,0.098180,0.035671,-0.210794,-0.004706,-0.318188,-0.141389,-0.153371,-0.012531,-0.228365,0.001537,-0.116103,0.064897,0.008819,0.117035,-0.350797,0.220972,-0.163182,-0.190531,0.127048,0.421333,-0.146783,-0.109759,0.097357,0.102098,-0.064166,-0.323645,-0.150754,0.236022,0.195547,0.280790,-0.347566,-0.112507,-0.190743,-0.159941,2,"[0.24856506, -0.19633645, 0.06935101, -0.20951..."
4,0,predicts that the current rate there likely in...,0.327024,-0.162196,0.143518,-0.192900,0.083366,-0.133948,0.095954,-0.087004,0.059492,-0.124035,0.145824,-0.162009,-0.277149,0.157513,0.132287,-0.018880,-0.231086,-0.015312,0.149329,0.040641,-0.112284,-0.197858,0.296189,0.010211,0.124315,0.120447,-0.179352,-0.105709,0.115390,-0.032078,-0.075978,-0.162349,0.16558

**Penentuan Cluster Zero**

In [ ]:
cluster_zero = df_final.loc[df_final['cluster'] == 0]
cluster_zero

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
9,1,ban that contains and octinoxate went into eff...,0.302793,-0.181907,0.121424,-0.203062,0.071630,-0.178152,0.185280,0.017518,0.107369,-0.105627,0.117441,-0.115476,-0.247916,0.188537,0.140233,-0.092141,-0.166991,-0.069744,-0.009424,0.049695,-0.182618,-0.144958,0.326874,0.029355,-0.062521,0.071563,-0.144517,-0.045385,0.003238,0.006833,-0.104029,-0.231626,0.248881,0.134598,0.095761,-0.112535,0.129669,0.090990,...,-0.136156,0.095191,0.378833,0.069909,0.129090,0.053896,-0.200216,0.053939,-0.203085,-0.175185,-0.186311,-0.089428,-0.253796,0.026716,0.054167,0.086669,-0.012585,0.115283,-0.242721,0.250262,-0.116027,-0.143270,0.038093,0.456092,-0.043376,-0.089012,0.122771,0.089729,-0.127397,-0.374197,-0.157137,0.043986,0.156776,0.366096,-0.245911,-0.088822,-0.144332,-0.311861,0,"[0.23149328, -0.21284643, 0.07621303, -0.18721..."
11,3,going special dive today with the coral restor...,0.210161,-0.242181,0.115292,-0.182452,0.089659,-0.157846,0.138589,-0.070869,0.152072,-0.061508,0.043026,-0.077374,-0.256662,0.150643,0.027947,-0.067156,-0.237477,-0.033785,0.095111,0.083271,-0.107192,-0.228901,0.364123,0.006331,-0.022224,0.137639,-0.175026,-0.152462,0.091893,-0.108718,0.067768,-0.279526,0.179478,0.175047,0.209393,-0.105697,0.202239,0.014362,...,-0.141834,0.124992,0.404100,-0.143106,0.067111,0.052692,-0.181823,0.054450,-0.368692,-0.130486,-0.095811,-0.052461,-0.247490,-0.110613,-0.059268,0.078086,-0.020808,0.178317,-0.331033,0.344838,-0.138167,-0.156892,0.198410,0.437335,-0.085882,-0.130380,0.115997,0.060428,-0.122470,-0.416568,-0.165477,0.204535,0.184196,0.228541,-0.301749,-0.091670,-0.131790,-0.249790,0,"[0.23149328, -0.21284643, 0.07621303, -0.18721..."
15,1,need clear when sediment pollutants enter the ...,0.306586,-0.210613,0.128501,-0.208318,0.059117,-0.187038,0.045769,-0.092999,0.066084,-0.090514,0.059418,-0.106821,-0.252891,0.150858,0.104651,-0.072194,-0.232208,-0.024900,0.091307,-0.004698,-0.081443,-0.169344,0.301827,0.031521,0.090036,0.128019,-0.158977,-0.094289,0.089923,-0.026804,-0.037133,-0.303738,0.201260,0.131697,0.147486,-0.187661,0.142557,0.096939,...,-0.173948,0.133134,0.418109,-0.057136,0.099311,0.054363,-0.218693,0.044237,-0.346516,-0.195280,-0.148677,-0.019494,-0.314895,-0.026196,-0.057476,0.055664,-0.068454,0.127015,-0.299125,0.248920,-0.106008,-0.167309,0.152437,0.350820,-0.067048,-0.104612,0.118195,0.128539,-0.058438,-0.352222,-0.225412,0.160007,0.184858,0.227139,-0.166235,-0.059018,-0.076760,-0.219962,0,"[0.23149328, -0.21284643, 0.07621303, -0.18721..."
16,4,would encourage people visit the following web...,0.269654,-0.238124,0.035636,-0.173958,0.103685,-0.193733,0.060105,-0.093762,0.148399,-0.122102,-0.076806,-0.085456,-0.273267,0.135272,0.183354,-0.082589,-0.211376,-0.051936,-0.041557,0.020257,-0.069789,-0.162029,0.425586,0.028125,0.021675,0.128732,-0.163965,-0.058163,0.048665,-0.087008,0.015136,-0.224221,0.188764,0.197676,0.168015,-0.173545,0.210055,0.090311,...,-0.202890,0.120342,0.413234,-0.193837,0.064041,0.077959,-0.220164,0.093874,-0.330596,-0.168613,-0.172820,0.004152,-0.234717,-0.070680,0.030817,0.038397,-0.022091,0.207865,-0.264366,0.284841,-0.135933,-0.187976,0.139043,0.371459,-0.054623,-0.146079,0.005073,0.127612,-0.115561,-0.274734,-0.142335,0.177141,0.113117,0.215425,-0.347731,-0.144458,-0.102695,-0.216414,0,"[0.23149328, -0.21284643, 0.07621303, -0.18721..."
33,1,research conducted the dmc and was recently pu...,0.225351,-0.227822,0.110879,-0.204395,0.098630,-0.131715,0.169236,-0.070152,0.096566,-0.126515,0.013766,-0.070367,-0.233798,0.168202,0.173893,-0.136638,-0.275884,-0.061822,0.052420,0.046331,-0.122889,-0.211899,0.345215,-0.036417,0.014756,0.091333,-0.151752,-0.082739,0.061156,-0.011555,-0.036374,-0.287506,0.

In [ ]:
cluster_zero['label'].value_counts()
# cluster 0 = Cluster Buzz

3    113
0     53
1     34
4     20
2     17
Name: label, dtype: int64

In [ ]:
counts_cluster_zero = cluster_zero['label'].value_counts().tolist()
counts_cluster_zero

[113, 53, 34, 20, 17]

In [ ]:
buzz_cluster_zero = (counts_cluster_zero[0] / len(cluster_zero) * 100)
desirability_cluster_zero = (counts_cluster_zero[1] / len(cluster_zero) * 100)
enabling_context_cluster_zero = (counts_cluster_zero[2] / len(cluster_zero) * 100)
invitation_cluster_zero = (counts_cluster_zero[3] / len(cluster_zero) * 100)
can_do_cluster_zero = (counts_cluster_zero[4] / len(cluster_zero) * 100)

In [ ]:
print("Cluster Zero Percentage\n Desirability : %.2f\n Enabling Context : %.2f\n Can Do : %.2f\n Buzz : %.2f\n Invitation : %.2f\n" % (desirability_cluster_zero,enabling_context_cluster_zero,can_do_cluster_zero,buzz_cluster_zero,invitation_cluster_zero))

Cluster Zero Percentage
 Desirability : 22.36
 Enabling Context : 14.35
 Can Do : 7.17
 Buzz : 47.68
 Invitation : 8.44



**Penentuan Cluster One**

In [ ]:
cluster_one = df_final.loc[df_final['cluster'] == 1]
cluster_one

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
4,0,predicts that the current rate there likely in...,0.327024,-0.162196,0.143518,-0.192900,0.083366,-0.133948,0.095954,-0.087004,0.059492,-0.124035,0.145824,-0.162009,-0.277149,0.157513,0.132287,-0.018880,-0.231086,-0.015312,0.149329,0.040641,-0.112284,-0.197858,0.296189,0.010211,0.124315,0.120447,-0.179352,-0.105709,0.115390,-0.032078,-0.075978,-0.162349,0.165580,0.177975,0.163704,-0.180910,0.082442,-0.008368,...,-0.153356,0.123537,0.425206,-0.086512,0.021996,0.053116,-0.244984,0.018976,-0.294460,-0.099158,-0.194668,0.046652,-0.308757,0.006832,-0.028352,-0.083012,-0.078689,0.221611,-0.303508,0.343517,-0.009814,-0.202835,0.162696,0.347850,0.041460,-0.128841,0.004591,0.099074,-0.143205,-0.320565,-0.041462,0.164817,0.163652,0.267573,-0.326143,0.051704,-0.190608,-0.176170,1,"[0.27742666, -0.17560728, 0.10773915, -0.19093..."
5,0,warming the ocean affects marine species cause...,0.276056,-0.204509,0.119345,-0.202238,0.047280,-0.173294,0.028553,-0.160531,0.115618,-0.082796,0.088808,-0.145281,-0.240652,0.128059,0.106973,-0.093626,-0.257584,0.000580,0.100056,0.069156,-0.128392,-0.199253,0.286176,0.019335,0.014647,0.138557,-0.219332,-0.110299,0.097884,-0.075593,-0.063908,-0.159737,0.142082,0.097608,0.159071,-0.252935,0.107512,0.051782,...,-0.193161,0.160220,0.381955,-0.014640,0.060248,0.067309,-0.297537,0.035888,-0.354320,-0.174411,-0.111734,-0.010421,-0.224346,0.008542,-0.114102,-0.014574,-0.065007,0.118491,-0.292315,0.279662,-0.089028,-0.277707,0.183987,0.355824,-0.051621,-0.165440,0.070846,0.064685,-0.216699,-0.364950,-0.103346,0.172426,0.129497,0.318646,-0.332814,0.031181,-0.154575,-0.118237,1,"[0.27742666, -0.17560728, 0.10773915, -0.19093..."
10,1,research reveals genetic response ocean warmin...,0.297440,-0.173013,0.129633,-0.182747,0.096588,-0.140084,0.079786,-0.103149,0.114765,-0.110723,0.007900,-0.086512,-0.268214,0.113311,0.162191,-0.116844,-0.187376,-0.016829,0.136294,0.037754,-0.160914,-0.196405,0.277107,0.036947,0.083491,0.135392,-0.147905,-0.046033,0.040568,-0.094677,-0.045905,-0.172682,0.130317,0.296624,0.199968,-0.163486,0.139458,0.026970,...,-0.145461,0.102110,0.424984,-0.070870,0.039490,0.042096,-0.259013,0.083870,-0.302123,-0.107459,-0.131387,-0.016754,-0.268337,0.016693,0.038458,0.043662,-0.056670,0.198677,-0.290071,0.373783,-0.075690,-0.191884,0.174595,0.327877,0.013904,-0.160377,0.102032,0.146935,-0.143250,-0.358263,-0.155973,0.153064,0.096072,0.261391,-0.345924,-0.035839,-0.195790,-0.213976,1,"[0.27742666, -0.17560728, 0.10773915, -0.19093..."
14,3,owe our children separate atmospheric climate ...,0.289432,-0.243320,0.143229,-0.189556,0.166839,-0.151868,0.010691,-0.066238,0.178216,-0.041226,-0.000934,-0.097317,-0.265572,0.121119,0.102791,-0.090561,-0.202900,-0.015345,0.087061,0.119630,-0.109894,-0.213642,0.335293,-0.006307,-0.010843,0.174558,-0.185401,-0.118323,0.032071,0.004642,-0.019018,-0.249236,0.172659,0.158086,0.312582,-0.214533,0.249103,0.033166,...,-0.138113,0.115540,0.489028,-0.078809,0.155752,0.012406,-0.281554,0.109431,-0.323833,-0.129269,-0.166620,-0.005685,-0.359811,-0.082565,-0.013466,0.116128,0.037755,0.238563,-0.262403,0.353330,-0.077970,-0.214295,0.170910,0.424341,0.015665,-0.113265,0.058991,0.141564,-0.234312,-0.293168,-0.134121,0.250063,0.120816,0.214365,-0.289813,-0.034083,-0.106689,-0.226871,1,"[0.27742666, -0.17560728, 0.10773915, -0.19093..."
18,1,wonderful see cover current sea anemone tentac...,0.264463,-0.199589,0.135287,-0.165760,0.105710,-0.125817,0.043070,0.011565,0.110518,-0.056765,0.053120,-0.191530,-0.283270,0.155009,0.182917,-0.039094,-0.242608,0.052039,-0.006896,0.065389,-0.104170,-0.200011,0.340542,0.018999,0.051621,0.056241,-0.167512,-0.100197,0.166025,-0.119566,0.022382,-0.195336,0.178394,0

In [ ]:
cluster_one['label'].value_counts()
#Cluster 1 = Cluster Desirability

0    114
1    113
3     15
2      6
4      2
Name: label, dtype: int64

In [ ]:
counts_cluster_one = cluster_one['label'].value_counts().tolist()
counts_cluster_one

[114, 113, 15, 6, 2]

In [ ]:
desirability_cluster_one = (counts_cluster_one[0] / len(cluster_one) * 100)
enabling_context_cluster_one = (counts_cluster_one[1] / len(cluster_one) * 100)
buzz_cluster_one = (counts_cluster_one[2] / len(cluster_one) * 100)
can_do_cluster_one = (counts_cluster_one[3] / len(cluster_one) * 100)
invitation_cluster_one = (counts_cluster_one[4] / len(cluster_one) * 100)

In [ ]:
print("Cluster One Percentage\n Desirability : %.2f\n Enabling Context : %.2f\n Can Do : %.2f\n Buzz : %.2f\n Invitation : %.2f\n" % (desirability_cluster_one,enabling_context_cluster_one,can_do_cluster_one,buzz_cluster_one,invitation_cluster_one))

Cluster One Percentage
 Desirability : 45.60
 Enabling Context : 45.20
 Can Do : 2.40
 Buzz : 6.00
 Invitation : 0.80



**Penentuan Cluster Two**




In [ ]:
cluster_two = df_final.loc[df_final['cluster'] == 2]
cluster_two

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
3,0,coral bleaching sad because they just there vi...,0.196565,-0.253519,-0.023528,-0.191615,0.086647,-0.220587,-0.055594,-0.048283,0.131703,-0.094454,-0.040577,-0.078372,-0.108059,0.052253,0.228489,-0.088540,-0.319645,-0.121290,0.046943,0.023376,-0.061790,-0.173290,0.351315,-0.008922,0.000887,0.142431,-0.187891,-0.083845,0.031222,-0.001472,-0.017065,-0.215411,0.114584,0.153335,0.115404,-0.104232,0.172777,0.120475,...,-0.234050,0.184041,0.429846,-0.131488,0.098180,0.035671,-0.210794,-0.004706,-0.318188,-0.141389,-0.153371,-0.012531,-0.228365,0.001537,-0.116103,0.064897,0.008819,0.117035,-0.350797,0.220972,-0.163182,-0.190531,0.127048,0.421333,-0.146783,-0.109759,0.097357,0.102098,-0.064166,-0.323645,-0.150754,0.236022,0.195547,0.280790,-0.347566,-0.112507,-0.190743,-0.159941,2,"[0.24856506, -0.19633645, 0.06935101, -0.20951..."
6,1,happening coral bleaching,0.233859,-0.185903,0.106083,-0.220513,0.063269,-0.158282,0.118587,-0.088741,0.201536,-0.107701,-0.070209,-0.015169,-0.204367,0.052346,0.180418,-0.059748,-0.220492,-0.101789,0.120386,0.064623,-0.157866,-0.201309,0.375666,0.002774,0.019217,0.131598,-0.176569,-0.054951,0.026446,-0.065052,-0.002227,-0.159360,0.088386,0.212903,0.173816,-0.077144,0.177521,0.108918,...,-0.208704,0.107102,0.412636,-0.131903,0.104709,0.074032,-0.284026,0.069789,-0.282880,-0.116154,-0.176889,0.010750,-0.197039,-0.017864,-0.034870,0.100573,-0.045097,0.202844,-0.284905,0.306340,-0.124507,-0.191761,0.190635,0.360901,-0.111013,-0.095275,0.072618,0.135722,-0.135883,-0.306383,-0.170318,0.127092,0.115386,0.265540,-0.371387,-0.047900,-0.130243,-0.190573,2,"[0.24856506, -0.19633645, 0.06935101, -0.20951..."
21,1,major discovery helps explain coral bleaching ...,0.272583,-0.221004,0.058090,-0.197673,0.049976,-0.164341,0.073875,-0.122522,0.162300,-0.104671,-0.026134,-0.041452,-0.215234,0.104096,0.175341,-0.062376,-0.147038,-0.074132,0.078482,0.095963,-0.141200,-0.223586,0.344359,-0.014822,0.074785,0.163945,-0.119161,-0.069402,0.017797,-0.047281,0.005148,-0.199331,0.106951,0.182054,0.193054,-0.145500,0.172569,0.149761,...,-0.206501,0.121393,0.410844,-0.118803,0.084282,0.055782,-0.323210,0.059877,-0.274861,-0.074595,-0.140076,-0.011007,-0.229690,-0.007074,-0.018300,0.095396,-0.040660,0.220416,-0.267233,0.296440,-0.131652,-0.190371,0.214769,0.359999,-0.086702,-0.123733,0.066113,0.109677,-0.102526,-0.280468,-0.181392,0.168686,0.159611,0.233570,-0.377532,-0.047444,-0.122983,-0.142339,2,"[0.24856506, -0.19633645, 0.06935101, -0.20951..."
22,2,get know more about the latest coral restorati...,0.222656,-0.121764,0.102878,-0.179216,0.048802,-0.139217,0.061353,-0.142851,0.213278,-0.137485,-0.064531,-0.130524,-0.238143,0.100117,0.208028,-0.019957,-0.200662,-0.064375,0.140678,0.108445,-0.169983,-0.184464,0.352228,-0.030883,-0.070939,0.083024,-0.232027,-0.035080,0.059446,-0.144245,-0.053069,-0.216815,0.122308,0.144431,0.214558,-0.137963,0.187434,0.114263,...,-0.138543,0.096148,0.397720,-0.081798,0.088748,0.022345,-0.275965,0.031154,-0.279139,-0.167309,-0.123496,-0.032314,-0.233372,-0.022447,-0.010511,0.044310,0.024823,0.190451,-0.310023,0.278355,-0.131567,-0.216802,0.207056,0.353577,-0.041768,-0.103267,0.022621,0.065613,-0.179148,-0.312263,-0.167021,0.164170,0.109599,0.285800,-0.366233,-0.006926,-0.087312,-0.235706,2,"[0.24856506, -0.19633645, 0.06935101, -0.20951..."
23,0,real,0.242023,-0.176803,0.070429,-0.218223,0.063311,-0.147885,0.113666,-0.101325,0.206389,-0.092175,-0.087881,0.020331,-0.203266,0.063431,0.211946,-0.065166,-0.221733,-0.108194,0.091307,0.067876,-0.168313,-0.190191,0.391801,0.015418,-0.001441,0.143689,-0.174669,-0.062302,0.004656,-0.024967,0.003526,-0.187121,0.099814,0.191794,0.193036,-0.086751,0.166320,0.141708,...,-0.20

In [ ]:
cluster_two['label'].value_counts()
#Cluster 2 = Cluster Enabling Context

1    109
0     96
3     75
4     22
2     11
Name: label, dtype: int64

In [ ]:
counts_cluster_two = cluster_two['label'].value_counts().tolist()
counts_cluster_two

[109, 96, 75, 22, 11]

In [ ]:
desirability_cluster_two = (counts_cluster_two[1] / len(cluster_two) * 100)
enabling_context_cluster_two = (counts_cluster_two[0] / len(cluster_two) * 100)
buzz_cluster_two = (counts_cluster_two[2] / len(cluster_two) * 100)
can_do_cluster_two = (counts_cluster_two[4] / len(cluster_two) * 100)
invitation_cluster_two = (counts_cluster_two[3] / len(cluster_two) * 100)

In [ ]:
print("Cluster Two Percentage\n Desirability : %.2f\n Enabling Context : %.2f\n Can Do : %.2f\n Buzz : %.2f\n Invitation : %.2f\n" % (desirability_cluster_two,enabling_context_cluster_two,can_do_cluster_two,buzz_cluster_two,invitation_cluster_two))

Cluster Two Percentage
 Desirability : 30.67
 Enabling Context : 34.82
 Can Do : 3.51
 Buzz : 23.96
 Invitation : 7.03



**Penentuan Cluster Three**

In [ ]:
cluster_three = df_final.loc[df_final['cluster'] == 3]
cluster_three

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
1,1,new insights into coral bleaching internationa...,0.278989,-0.210553,0.116534,-0.220855,0.026345,-0.185116,0.102590,-0.142175,0.066128,-0.152199,-0.003948,-0.127940,-0.262440,0.150455,0.154574,-0.015205,-0.147123,0.033080,0.117147,0.082415,-0.193111,-0.194614,0.289233,-0.045297,0.034218,0.129791,-0.166426,-0.094949,0.109077,-0.081011,-0.022905,-0.255659,0.114677,0.213910,0.165936,-0.204131,0.134564,0.064170,...,-0.130318,0.158661,0.368667,-0.099549,0.056852,0.038233,-0.229869,0.054649,-0.280238,-0.200504,-0.132204,0.040076,-0.213502,-0.052318,-0.019564,0.085647,0.006973,0.123810,-0.326739,0.272688,-0.123412,-0.134248,0.159430,0.330009,0.030360,-0.118556,0.050030,0.047144,-0.147236,-0.331968,-0.149588,0.110895,0.127745,0.284505,-0.328791,0.033071,-0.195695,-0.169238,3,"[0.27022475, -0.20869835, 0.082921416, -0.2030..."
2,3,friday continued coral bleaching documentation...,0.292273,-0.185109,0.103384,-0.195429,0.099492,-0.132130,0.051576,-0.111612,0.108165,-0.140106,0.050763,-0.107536,-0.202583,0.039158,0.162155,-0.040363,-0.202544,-0.043050,0.133335,0.085618,-0.123567,-0.227499,0.301947,-0.018139,-0.006174,0.150228,-0.108965,-0.049512,0.111773,-0.091334,0.001697,-0.189163,0.086513,0.200552,0.130424,-0.175857,0.161722,0.030261,...,-0.229967,0.148397,0.412870,-0.160140,0.041154,0.043219,-0.246439,0.076627,-0.274997,-0.134232,-0.117466,0.048612,-0.157647,0.045113,-0.059363,0.082873,-0.038386,0.259776,-0.338416,0.241494,-0.088733,-0.111480,0.178569,0.369975,-0.070284,-0.110966,0.084464,0.102760,-0.082984,-0.293506,-0.167755,0.122283,0.081358,0.308439,-0.346580,-0.050084,-0.225804,-0.227144,3,"[0.27022475, -0.20869835, 0.082921416, -0.2030..."
7,4,going the biggest coral restoration event happ...,0.211341,-0.170488,0.117109,-0.145355,0.077081,-0.156249,0.089607,-0.175697,0.250045,-0.112441,-0.074138,-0.129633,-0.285934,0.081036,0.118326,-0.083827,-0.142216,-0.052036,0.040303,0.083212,-0.072228,-0.226718,0.289834,0.037513,-0.031164,0.129399,-0.252153,-0.056424,-0.030564,-0.070300,0.010100,-0.201271,0.125670,0.159346,0.274050,-0.184875,0.143889,0.079248,...,-0.133023,0.183987,0.407491,-0.186861,0.093786,-0.007352,-0.308978,0.009589,-0.334257,-0.135311,-0.047273,0.018112,-0.210061,-0.092890,-0.050255,0.102627,0.011551,0.211647,-0.295133,0.207356,-0.102693,-0.144457,0.193226,0.355389,-0.006117,-0.162299,-0.002737,0.063247,-0.255003,-0.315541,-0.065468,0.148291,0.156737,0.246241,-0.362413,-0.012624,-0.130832,-0.197387,3,"[0.27022475, -0.20869835, 0.082921416, -0.2030..."
8,4,january 30th 2021 guest lecture franziska ston...,0.282701,-0.101572,0.159982,-0.235290,-0.003450,-0.205923,-0.036873,-0.127074,0.163583,-0.106472,0.040189,-0.187498,-0.259482,0.178982,0.182048,-0.058230,-0.222654,0.090636,0.077255,0.064476,-0.127555,-0.238571,0.282734,-0.018892,0.023948,0.145860,-0.207479,-0.092634,0.107404,-0.041792,-0.089958,-0.226723,0.180865,0.241400,0.177789,-0.250981,0.123588,0.176426,...,-0.141216,0.204007,0.382163,-0.075585,0.007933,-0.077689,-0.331764,-0.038570,-0.233141,-0.209052,-0.041151,-0.087871,-0.249297,0.019838,-0.031975,0.049125,0.049296,0.192215,-0.226584,0.261811,-0.144198,-0.127153,0.223370,0.390596,-0.112909,-0.144622,-0.012209,0.016268,-0.193976,-0.339859,-0.088491,0.143470,0.111373,0.308487,-0.271675,0.045909,-0.210736,-0.209602,3,"[0.27022475, -0.20869835, 0.082921416, -0.2030..."
12,1,american lobster postlarvae alter gene regulat...,0.317362,-0.236568,0.132554,-0.207529,0.088546,-0.180980,0.110550,-0.158905,0.093462,-0.089559,-0.026201,-0.032451,-0.265808,0.128748,0.190690,-0.068462,-0.180909,-0.091242,0.081110,0.110926,-0.158870,-0.192082,0.336554,-0.048397,0.090106,0.154166,-0.139832,-0.076667,0.028148,-0.040052,-0.044852,-0.153723,0

In [ ]:
cluster_three['label'].value_counts()
#Cluster 3 = Cluster Enabling Context

1    98
3    67
0    53
2    14
4    11
Name: label, dtype: int64

In [ ]:
counts_cluster_three = cluster_three['label'].value_counts().tolist()
counts_cluster_three

[98, 67, 53, 14, 11]

In [ ]:
desirability_cluster_three = (counts_cluster_three[2] / len(cluster_three) * 100)
enabling_context_cluster_three = (counts_cluster_three[0] / len(cluster_three) * 100)
buzz_cluster_three = (counts_cluster_three[1] / len(cluster_three) * 100)
can_do_cluster_three = (counts_cluster_three[3] / len(cluster_three) * 100)
invitation_cluster_three = (counts_cluster_three[4] / len(cluster_three) * 100)

In [ ]:
print("Cluster Three Percentage\n Desirability : %.2f\n Enabling Context : %.2f\n Can Do : %.2f\n Buzz : %.2f\n Invitation : %.2f\n" % (desirability_cluster_three,enabling_context_cluster_three,can_do_cluster_three,buzz_cluster_three,invitation_cluster_three))

Cluster Three Percentage
 Desirability : 21.81
 Enabling Context : 40.33
 Can Do : 5.76
 Buzz : 27.57
 Invitation : 4.53



**Penentuan Cluster Four**

In [ ]:
cluster_four = df_final.loc[df_final['cluster'] == 4]
cluster_four

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
0,1,cyanobacteria called are microorganisms that c...,0.269467,-0.237420,0.135789,-0.192115,0.092847,-0.209697,0.151208,0.013293,0.062865,-0.040700,-0.029866,-0.065176,-0.253705,0.151779,0.113438,-0.026016,-0.196706,0.011119,0.023070,0.025161,-0.181920,-0.156438,0.293603,0.047054,-0.022497,0.095719,-0.257094,-0.014528,-0.010651,0.068333,-0.049718,-0.232437,0.338180,0.261451,0.188081,-0.174729,0.128956,0.033906,...,-0.146089,0.190156,0.335575,-0.025112,0.119993,0.064195,-0.212036,0.042518,-0.273705,-0.080256,-0.197115,-0.060150,-0.168341,0.013836,-0.037829,0.069539,-0.080841,0.045633,-0.299846,0.345259,-0.188063,-0.256294,0.115115,0.344267,-0.073296,-0.155676,0.168050,0.128368,-0.087861,-0.377813,-0.091366,0.112518,0.142527,0.185022,-0.214439,0.012216,-0.082393,-0.240312,4,"[0.26434752, -0.2272363, 0.108652465, -0.19666..."
31,1,lost its coral reefs 1998 not overfishing but ...,0.243102,-0.152948,0.140749,-0.220966,0.088364,-0.250552,0.064058,-0.123122,0.162791,-0.081509,0.099618,-0.043050,-0.180234,0.072605,0.156396,-0.055256,-0.149061,-0.087500,0.174587,0.168940,-0.202050,-0.160413,0.187805,-0.006547,-0.050129,0.192586,-0.131293,-0.077095,0.093949,0.014124,-0.154651,-0.151271,0.118443,0.198127,0.237314,-0.279522,0.095954,0.045679,...,-0.211211,0.112450,0.353112,-0.092859,0.028831,-0.040578,-0.204271,0.055605,-0.286512,-0.073526,0.020510,0.067023,-0.231986,0.060712,-0.090086,0.130516,-0.009152,0.231782,-0.296909,0.222326,-0.093110,-0.224621,0.209816,0.352793,-0.031145,-0.153933,0.090967,0.038273,-0.144427,-0.274121,-0.070175,0.142254,0.173430,0.295719,-0.421124,-0.069260,-0.148443,-0.161484,4,"[0.26434752, -0.2272363, 0.108652465, -0.19666..."
32,1,mangroves counteract adjacent tropical well fi...,0.247128,-0.099177,0.161028,-0.091492,0.100168,-0.188271,0.059252,-0.150255,0.164701,-0.010039,0.099010,-0.136216,-0.294692,0.150215,0.093672,0.013563,-0.164027,0.004641,0.018866,0.022878,-0.237306,-0.168566,0.198691,0.053749,0.036696,0.152877,-0.023319,-0.152140,0.123175,-0.014691,-0.047304,-0.162318,0.205368,0.266039,0.154764,-0.163534,0.115335,0.058710,...,-0.154590,0.096167,0.273443,-0.060895,0.082897,0.026378,-0.292293,0.063701,-0.347816,-0.098095,-0.111504,-0.022325,-0.228624,0.022074,-0.019573,0.078673,0.038051,0.153960,-0.268912,0.321436,-0.171189,-0.187328,0.212428,0.396347,-0.044682,-0.060106,0.120302,0.070704,-0.119435,-0.368466,-0.178195,0.146243,0.090733,0.321364,-0.190074,0.018070,-0.160906,-0.280514,4,"[0.26434752, -0.2272363, 0.108652465, -0.19666..."
35,1,ocean temperatures rising can also impact cora...,0.310083,-0.224828,0.154053,-0.183674,0.065103,-0.221297,0.089878,-0.126788,0.149830,-0.056755,0.063553,-0.084601,-0.223238,0.098694,0.147140,-0.031263,-0.161989,-0.047342,0.144773,0.131450,-0.162647,-0.240356,0.282244,0.035889,0.007925,0.205008,-0.166910,-0.180143,0.069942,-0.008675,-0.055211,-0.109781,0.182515,0.192698,0.198136,-0.234601,0.082074,0.029118,...,-0.142362,0.109509,0.399777,-0.096880,0.054670,0.018071,-0.270889,0.041369,-0.347332,-0.090730,-0.050277,0.020392,-0.315657,0.021322,-0.077914,0.075075,-0.026411,0.236654,-0.286992,0.262340,-0.120368,-0.218163,0.237490,0.352830,-0.019426,-0.142343,0.085577,0.082510,-0.184498,-0.365912,-0.072323,0.238063,0.150628,0.294663,-0.380550,-0.039380,-0.170925,-0.184790,4,"[0.26434752, -0.2272363, 0.108652465, -0.19666..."
55,1,neoteric marine biology research reveals furth...,0.368360,-0.190193,0.180307,-0.178567,0.019830,-0.189838,0.081044,-0.026620,0.039211,-0.059240,0.050287,-0.148704,-0.257786,0.215124,0.114992,-0.033183,-0.161014,0.070078,0.008717,0.056328,-0.210709,-0.205579,0.260487,0.046153,0.046125,0.150222,-0.216625,-0.081381,0.112127,-0.031868,-0.048696,-0.272719,0.213695,0.

In [ ]:
cluster_four['label'].value_counts()
#Cluster 4 = Cluster Enabling Context

1    75
0    57
3    38
2     5
4     4
Name: label, dtype: int64

In [ ]:
counts_cluster_four = cluster_four['label'].value_counts().tolist()
counts_cluster_four

[75, 57, 38, 5, 4]

In [ ]:
desirability_cluster_four = (counts_cluster_four[1] / len(cluster_four) * 100)
enabling_context_cluster_four = (counts_cluster_four[0] / len(cluster_four) * 100)
buzz_cluster_four = (counts_cluster_four[2] / len(cluster_four) * 100)
can_do_cluster_four = (counts_cluster_four[3] / len(cluster_four) * 100)
invitation_cluster_four = (counts_cluster_four[4] / len(cluster_four) * 100)

In [ ]:
print("Cluster Four Percentage\n Desirability : %.2f\n Enabling Context : %.2f\n Can Do : %.2f\n Buzz : %.2f\n Invitation : %.2f\n" % (desirability_cluster_four,enabling_context_cluster_four,can_do_cluster_four,buzz_cluster_four,invitation_cluster_four))

Cluster Four Percentage
 Desirability : 31.84
 Enabling Context : 41.90
 Can Do : 2.79
 Buzz : 21.23
 Invitation : 2.23

